<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/passaporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problema do Passaporte**

Amanda Topanotti Zanette (22100776)

Este notebook ilustra o básico de manipulação de dados em Julia e formulação de um problema de cobertura de conjuntos em JuMP

In [83]:
# Pacotes necessarios (se preciso, instale)
using CSV
using DataFrames
using JuMP
using HiGHS

Coloque o arquivo "passport-index-matrix.csv" dentro da pasta /content/data

O arquivo está disponível na página da disciplina: http://mtm.ufsc.br/~douglas/2025.2/MTM3531/

A seguir, leia o arquivo com o comando abaixo:

In [94]:
const DATA_DIR = joinpath(@__DIR__, "data")
passport_data = CSV.read(
    joinpath(DATA_DIR, "passport-index-matrix.csv"),
    DataFrames.DataFrame,
)

Row,Passport,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,DR Congo,Costa Rica,Ivory Coast,Croatia,Cuba,Cyprus,Czech Republic,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Swaziland,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,⋯
,String,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,⋯
1,Afghanistan,e-visa,visa required,visa required,visa on arrival,e-visa,visa required,visa required,e-visa,visa required,covid ban,visa required,e-visa,visa on arrival,visa required,visa required,visa required,visa required,e-visa,visa required,visa on arrival,visa required,e-visa,visa required,visa required,visa required,visa required,visa on arrival,e-visa,covid ban,visa required,visa on arrival,visa required,visa required,visa required,visa required,e-visa,visa on arrival,visa required,e-visa,visa required,e-visa,visa required,visa required,visa required,visa required,visa required,e-visa,21,visa required,visa required,visa required,visa required,visa required,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,visa required,visa required,visa required,visa required,e-visa,visa on arrival,visa on arrival,90,visa required,visa required,visa required,visa required,e-visa,covid ban,visa required,visa required,visa required,visa required,visa required,visa required,covid ban,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,⋯
2,Albania,-1,visa required,90,visa on arrival,180,visa required,180,e-visa,90,e-visa,visa required,e-visa,visa on arrival,28,30,90,visa required,e-visa,visa required,visa on arrival,90,e-visa,90,visa required,90,visa required,visa on arrival,e-visa,covid ban,visa required,visa on arrival,visa required,visa required,90,visa required,90,visa on arrival,visa required,e-visa,visa required,e-visa,90,visa required,90,90,90,e-visa,21,visa required,90,visa on arrival,90,visa required,visa required,90,visa required,e-visa,visa required,90,90,e-visa,90,360,90,visa required,90,visa required,visa required,e-visa,visa on arrival,90,90,visa required,14,90,90,e-visa,covid ban,e-visa,visa required,visa required,90,90,visa on arrival,visa required,visa required,visa required,e-visa,visa required,90,visa required,60,visa on arrival,90,visa required,e-visa,visa required,visa required,90,⋯
3,Algeria,e-visa,-1,visa required,visa on arrival,e-visa,visa required,visa requ

A 1a coluna de "passport_data" representa todos os passportes que podem ser obtidos. As colunas restantes indicam os destinos.

In [90]:
# quais paises estao incluidos na lista?
all_countries = passport_data[!, "Passport"]

199-element SentinelArrays.ChainedVector{String, Vector{String}}:
 "Afghanistan"
 "Albania"
 "Algeria"
 "Andorra"
 "Angola"
 "Antigua and Barbuda"
 "Argentina"
 "Armenia"
 "Australia"
 "Austria"
 "Azerbaijan"
 "Bahamas"
 "Bahrain"
 ⋮
 "United Arab Emirates"
 "United Kingdom"
 "United States"
 "Uruguay"
 "Uzbekistan"
 "Vanuatu"
 "Vatican"
 "Venezuela"
 "Vietnam"
 "Yemen"
 "Zambia"
 "Zimbabwe"

Verificação: será que todos os países de origem estão também listados como de destino?

In [105]:
from_countries = passport_data[!, "Passport"]
to_countries   = names(passport_data)[2:end]  # ignorando a primeira coluna com os nomes

only_in_from = setdiff(from_countries, to_countries)
only_in_to   = setdiff(to_countries, from_countries)

println("Apenas em from: ", only_in_from)
println("Apenas em to: ", only_in_to)

Apenas em from: String[]
Apenas em to: String[]


Todos os países estão listados, mas já pelo preview do DataFrame, é possível verificar que o Afeganistão está fora de ordem. Iremos reordenar as colunas e linhas. Observe que agora a diagonal é sempre -1.


In [106]:
all_countries_ordered = sort(union(from_countries, to_countries))

row_indices = [findfirst(==(c), from_countries) for c in all_countries_ordered]
passport_data = passport_data[row_indices, :]
col_indices = [findfirst(==(c), to_countries) + 1 for c in all_countries_ordered]

passport_data = hcat(passport_data[:, 1], passport_data[:, col_indices])
rename!(passport_data, 1 => "Passport")

Row,Passport,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Croatia,Cuba,Cyprus,Czech Republic,DR Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Ivory Coast,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,⋯
,String,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String7,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,String15,⋯
1,Afghanistan,-1,e-visa,visa required,visa required,visa on arrival,e-visa,visa required,visa required,e-visa,visa required,covid ban,visa required,e-visa,visa on arrival,visa required,visa required,visa required,visa required,e-visa,visa required,visa on arrival,visa required,e-visa,visa required,visa required,visa required,visa required,visa on arrival,e-visa,covid ban,visa required,visa on arrival,visa required,visa required,visa required,visa required,e-visa,visa on arrival,visa required,visa required,visa required,visa required,visa required,visa required,e-visa,visa required,e-visa,21,visa required,visa required,visa required,visa required,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,visa required,visa required,visa required,visa required,e-visa,visa on arrival,visa on arrival,90,visa required,visa required,visa required,visa required,e-visa,covid ban,visa required,visa required,visa required,visa required,visa required,e-visa,visa required,covid ban,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,e-visa,visa required,visa required,visa required,⋯
2,Albania,visa required,-1,visa required,90,visa on arrival,180,visa required,180,e-visa,90,e-visa,visa required,e-visa,visa on arrival,28,30,90,visa required,e-visa,visa required,visa on arrival,90,e-visa,90,visa required,90,visa required,visa on arrival,e-visa,covid ban,visa required,visa on arrival,visa required,visa required,90,visa required,90,visa on arrival,visa required,visa required,90,visa required,90,90,e-visa,90,e-visa,21,visa required,90,visa on arrival,90,visa required,visa required,90,e-visa,visa required,90,90,e-visa,90,360,90,visa required,90,visa required,visa required,e-visa,visa on arrival,90,90,visa required,14,90,90,e-visa,covid ban,e-visa,visa required,visa required,90,90,e-visa,visa on arrival,visa required,visa required,visa required,e-visa,visa required,90,visa required,60,visa on arrival,90,visa required,e-visa,visa required,visa required,90,⋯
3,Algeria,visa required,e-visa,-1,visa required,visa on arrival,e-visa,visa required,visa

Obtenha todos os países a partir das linhas da 1a coluna e, a seguir, obtenha uma matriz 199 x 199 cujas entradas (i,j) devem conter o requisito(código) para viajar com passaporte do país i para o país de destino j

In [121]:
raw_passport_matrix = Matrix(passport_data[:, Not("Passport")])

199×199 Matrix{Any}:
 "-1"             …  "e-visa"
 "visa required"     "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"     "90"
 "visa required"  …  "90"
 "visa required"     "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"  …  "visa on arrival"
 "visa required"     "90"
 "visa required"     "visa on arrival"
 ⋮                          ⋱  
 "visa required"     "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"  …  "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"     "90"
 "visa required"     "visa on arrival"
 "visa required"     "visa on arrival"
 "visa required"  …  "e-visa"
 "visa required"     "e-visa"
 "visa required"     "90"
 "visa required"     "-1"

A seguinte função verifica o código e retorna "true" quando a entrada é permitida "sem visto" ou quando o "visto é garantido na chegada", e "false" caso contrário.

In [122]:
function can_we_enter(code)
    if code == "visa free" || code == "visa on arrival" || code isa Int
        return true
    end
    try
        val = parse(Int, code)
        return true
    catch
        return false
    end
end

can_we_enter (generic function with 1 method)

O código a seguir cria uma matriz Booleana (0s e 1s) cuja entrada (i,j) indica se o detentor de passaporte do país i pode entrar no país j sem visto

In [123]:
passport_coverage = map(can_we_enter, raw_passport_matrix)

199×199 Matrix{Bool}:
 1  0  0  0  1  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  1  1  0  1  0  1  0  0  0     0  0  0  0  0  0  1  0  0  0  1  1
 0  0  1  0  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  1  0  1
 0  1  0  1  1  1  1  1  0  1  0  1  1     1  1  0  1  1  0  1  1  0  0  1  1
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  1
 0  1  0  1  1  1  0  0  0  1  0  1  0  …  0  1  0  0  1  0  1  1  0  0  1  1
 0  1  0  1  1  1  1  1  0  1  0  1  1     1  1  0  1  1  0  1  1  0  0  1  1
 0  1  0  0  1  1  1  1  0  0  0  1  0     0  0  0  1  1  0  0  0  0  0  0  1
 0  1  0  1  1  1  1  1  1  1  0  1  1     1  1  0  1  1  0  1  1  0  0  1  1
 0  1  0  1  1  1  1  1  0  1  0  1  1     1  1  0  1  1  0  1  1  0  0  1  1
 0  1  0  0  1  1  0  0  0  0  1  1  0  …  0  0  0  0  1  0  0  0  0  0  0  1
 0  1  0  1  1  1  0  0  0  1  0  1  0     1  1  1  1  1  0  1  0  0  0  1  1
 0  1  0  0  1  0  0  0  0  0  1  1  1    

Chegamos ao problema de otimização: qual o menor número de passaportes necessários para permitir entrada livre de visto a todos os países?

In [124]:
# Create a new JuMP model using the HiGHS solver.
model = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [125]:
n = length(all_countries)

# Defina um vetor de variáveis de decisão binárias, uma para cada passaporte que pode ser obtido.
@variable(model, x[1:n], Bin)

199-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]
 x[9]
 x[10]
 x[11]
 x[12]
 x[13]
 ⋮
 x[188]
 x[189]
 x[190]
 x[191]
 x[192]
 x[193]
 x[194]
 x[195]
 x[196]
 x[197]
 x[198]
 x[199]

In [126]:
# O objetivo é minimizar o número de passaportes necessários
@objective(model, Min, sum(x[i] for i in 1:n))

x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18] + x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] + [[...139 terms omitted...]] + x[170] + x[171] + x[172] + x[173] + x[174] + x[175] + x[176] + x[177] + x[178] + x[179] + x[180] + x[181] + x[182] + x[183] + x[184] + x[185] + x[186] + x[187] + x[188] + x[189] + x[190] + x[191] + x[192] + x[193] + x[194] + x[195] + x[196] + x[197] + x[198] + x[199]

In [128]:
# Restricões
for j in 1:n
    @constraint(model, sum(passport_coverage[i,j]*x[i] for i in 1:n) >= 1)
end

In [129]:
# Otimizar!
optimize!(model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 398 rows; 199 cols; 35544 nonzeros; 199 integer variables (199 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
147 rows, 188 cols, 8422 nonzeros  0s
28 rows, 64 cols, 280 nonzeros  0s
23 rows, 34 cols, 161 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   23 rows
   34 cols (22 binary, 12 integer, 0 implied int., 0 continuous, 0 domain fixed)
   161 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            

In [130]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

Quais os países que devem ser incluídos na solução porque requerem visto de todos os outros?

In [131]:
# soma cada coluna incluindo a diagonal
colsum_incl_diag = sum(passport_coverage, dims=1)[:]  # vetor de somas das colunas

# encontrar índices das colunas com exatamente um 1
must_include_indices = findall(colsum_incl_diag .== 1)

# nomes dos países correspondentes
must_include_countries = all_countries[must_include_indices]

println("Países cuja coluna tem apenas um 1:")
for c in must_include_countries
    println(" - ", c)
end

Países cuja coluna tem apenas um 1:
 - Afghanistan
 - Malta
 - Mexico
 - Mozambique
 - Nigeria
 - South Korea
 - Sri Lanka
 - Taiwan
 - Tonga
 - Turkmenistan
 - Vanuatu


Esta atividade deve ser entregue até **31/10/2025**.

Pode compartilhar o arquivo do Google Colab.